In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings
import itertools

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path+'../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb09 = pd.read_csv(path+'../sub/lgCV_0906.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [ ]:
#rnn27trn = pd.read_pickle('../../features/y_pred_trn.pkl')
#rnn27tst = pd.read_csv('../rnndhsub_2705A.csv.gz', compression='gzip')
#rnn27 = pd.concat([rnn27tst, rnn27trn.reset_index().rename(columns = {0:'deal_probability'})], axis = 0)
#rnn27.to_csv('../rnnCV_2705A.csv.gz', compression='gzip')

In [2]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
lgb09.rename(columns={'deal_probability': 'lgb09_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability': 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb09, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [3]:
#preds_df['difference_l02_r']=preds_df['lgb02A_preds'] - preds_df['rnn_preds']
#preds_df['difference_l02_r1']=preds_df['lgb02_preds'] - preds_df['rnn27_preds']
#preds_df['difference_l02_m']=preds_df['lgb02_preds'] - preds_df['mlp_preds']
#preds_df['difference_r_m']=preds_df['rnn27_preds'] - preds_df['mlp_preds']
#preds_df['difference_l02_l02A']=preds_df['lgb02_preds'] - preds_df['lgb02A_preds']
#preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
pred_cols = [col for col in preds_df.columns if ('_preds' in col) \
             and ('lgb27' not in col) and ('lgb02_' not in col)]

preds_df['preds_sum'] = preds_df[pred_cols].sum(axis=1)
pred_cols

['rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb09_preds',
 'lgb02A_preds',
 'rnn27_preds']

In [4]:
preds_df['price'].fillna(-1,inplace=True)
preds_df['max'] = np.max(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in pred_cols]),axis=0)

In [5]:
import itertools
for p1, p2 in itertools.combinations(pred_cols, 2):
    print('difference_%s__%s'%(p1,p2))
    preds_df['difference_%s__%s'%(p1,p2)] = preds_df[p2] - preds_df[p1]

difference_rnn_preds__mlp_preds
difference_rnn_preds__lgb31_preds
difference_rnn_preds__lgb09_preds
difference_rnn_preds__lgb02A_preds
difference_rnn_preds__rnn27_preds
difference_mlp_preds__lgb31_preds
difference_mlp_preds__lgb09_preds
difference_mlp_preds__lgb02A_preds
difference_mlp_preds__rnn27_preds
difference_lgb31_preds__lgb09_preds
difference_lgb31_preds__lgb02A_preds
difference_lgb31_preds__rnn27_preds
difference_lgb09_preds__lgb02A_preds
difference_lgb09_preds__rnn27_preds
difference_lgb02A_preds__rnn27_preds


In [6]:

gc.collect()

174

In [7]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb02A_preds,Unnamed: 0,rnn27_preds,...,difference_mlp_preds__lgb31_preds,difference_mlp_preds__lgb09_preds,difference_mlp_preds__lgb02A_preds,difference_mlp_preds__rnn27_preds,difference_lgb31_preds__lgb09_preds,difference_lgb31_preds__lgb02A_preds,difference_lgb31_preds__rnn27_preds,difference_lgb09_preds__lgb02A_preds,difference_lgb09_preds__rnn27_preds,difference_lgb02A_preds__rnn27_preds
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0.090855,0.109624,0.109617,0,0.068331,...,-0.008004,0.018401,0.018394,-0.022892,0.026405,0.026398,-0.014888,-0.000007,-0.041293,-0.041287
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,0.119609,0.157466,0.160045,1,0.070638,...,-0.053871,-0.054378,-0.051800,-0.141206,-0.000507,0.002071,-0.087335,0.002578,-0.086828,-0.089406
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,0.258980,0.245236,0.241336,2,0.161924,...,-0.009531,0.008656,0.004757,-0.074655,0.018188,0.014288,-0.065124,-0.003900,-0.083311,-0.079412
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,0.284130,0.289101,0.336456,3,0.278804,...,-0.152251,-0.094598,-0.047243,-0.104895,0.057654,0.105008,0.047356,0.047355,-0.010297,-0.057652
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,0.430581,0.511145,0.421989,4,0.350284,...,-0.008094,0.090488,0.001332,-0.070373,0.098582,0.009426,-0.062278,-0.089156,-0.160860,-0.071704


### Train scores

In [8]:
pred_cols = [col for col in preds_df.columns if ('preds' in col) and ('difference' not in col)]
pred_cols

['lgb27_preds',
 'rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb02_preds',
 'lgb09_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'preds_sum']

In [9]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE lgb09_preds:  0.21581543265040168
RMSE lgb02A_preds:  0.21627136604335995
RMSE rnn27_preds:  0.21698040871120608


In [10]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [11]:
scores = []
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
5,lgb09_preds,0.215514,0.215678,0.218418,0.216073,0.212293
6,lgb02A_preds,0.216081,0.216051,0.218825,0.216562,0.212741
7,rnn27_preds,0.218097,0.218115,0.214982,0.218651,0.214379


### Correlations in test and train

In [12]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb02A_preds,rnn27_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.988682,0.978331,0.983143,0.916804
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.913307,0.908867,0.911443,0.985360
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.916343,0.909521,0.913600,0.925010
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.991984,0.981891,0.986851,0.914204
lgb02_preds,0.988682,0.913307,0.916343,0.991984,1.000000,0.982596,0.987434,0.916726
lgb09_preds,0.978331,0.908867,0.909521,0.981891,0.982596,1.000000,0.988708,0.912375
lgb02A_preds,0.983143,0.911443,0.913600,0.986851,0.987434,0.988708,1.000000,0.914911
rnn27_preds,0.916804,0.985360,0.925010,0.914204,0.916726,0.912375,0.914911,1.000000


In [13]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb02A_preds,rnn27_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.989502,0.979729,0.984742,0.930692
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.927867,0.926158,0.928496,0.988198
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.921927,0.918064,0.922121,0.935069
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.993168,0.982874,0.988043,0.931468
lgb02_preds,0.989502,0.927867,0.921927,0.993168,1.000000,0.984427,0.989577,0.931447
lgb09_preds,0.979729,0.926158,0.918064,0.982874,0.984427,1.000000,0.989106,0.929748
lgb02A_preds,0.984742,0.928496,0.922121,0.988043,0.989577,0.989106,1.000000,0.932071
rnn27_preds,0.930692,0.988198,0.935069,0.931468,0.931447,0.929748,0.932071,1.000000


In [14]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
cols = list(set(cols))

In [15]:
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [16]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [17]:
len(train_X)

1202739

In [18]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# valid_1's rmse: 0.213698

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220422	valid_1's rmse: 0.220807
[200]	training's rmse: 0.214078	valid_1's rmse: 0.214718
[300]	training's rmse: 0.212919	valid_1's rmse: 0.213779
[400]	training's rmse: 0.212533	valid_1's rmse: 0.213603
[500]	training's rmse: 0.212296	valid_1's rmse: 0.213547
[600]	training's rmse: 0.212106	valid_1's rmse: 0.213519
[700]	training's rmse: 0.211939	valid_1's rmse: 0.213504
[800]	training's rmse: 0.2118	valid_1's rmse: 0.213492
[900]	training's rmse: 0.211673	valid_1's rmse: 0.213483
[1000]	training's rmse: 0.211544	valid_1's rmse: 0.213474
[1100]	training's rmse: 0.211419	valid_1's rmse: 0.213467
[1200]	training's rmse: 0.211301	valid_1's rmse: 0.213459
[1300]	training's rmse: 0.211185	valid_1's rmse: 0.213454
[1400]	training's rmse: 0.211072	valid_1's rmse: 0.213447
[1500]	training's rmse: 0.210969	valid_1's rmse: 0.213443
[1600]	training's rmse: 0.210856	valid_1's rmse: 0.213439
[1700]	training's rms

In [19]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(10268, 'param_1'),
 (5818, 'param_2'),
 (5344, 'region'),
 (2813, 'category_name'),
 (2230, 'difference_rnn_preds__rnn27_preds'),
 (2075, 'rnn27_preds'),
 (2036, 'difference_lgb31_preds__lgb09_preds'),
 (1990, 'lgb09_preds'),
 (1957, 'avg'),
 (1804, 'difference_lgb09_preds__lgb02A_preds'),
 (1802, 'mlp_preds'),
 (1640, 'min'),
 (1624, 'difference_rnn_preds__mlp_preds'),
 (1585, 'difference_mlp_preds__lgb31_preds'),
 (1581, 'difference_lgb31_preds__lgb02A_preds'),
 (1558, 'difference_mlp_preds__rnn27_preds'),
 (1526, 'lgb02_preds'),
 (1520, 'price'),
 (1428, 'difference_mlp_preds__lgb02A_preds'),
 (1422, 'difference_lgb31_preds__rnn27_preds'),
 (1376, 'difference_mlp_preds__lgb09_preds'),
 (1324, 'difference_rnn_preds__lgb09_preds'),
 (1293, 'difference_lgb02A_preds__rnn27_preds'),
 (1261, 'lgb27_preds'),
 (1198, 'difference_lgb09_preds__rnn27_preds'),
 (1163, 'rnn_preds'),
 (1160, 'difference_rnn_preds__lgb02A_preds'),
 (1125, 'difference_rnn_preds__lgb31_preds'),
 (1114, 'std'),
 (1

In [20]:
n_estimators = 2146
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [21]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220509
[200]	training's rmse: 0.214214
[300]	training's rmse: 0.213087
[400]	training's rmse: 0.212734
[500]	training's rmse: 0.212522
[600]	training's rmse: 0.21235
[700]	training's rmse: 0.212204
[800]	training's rmse: 0.212082
[900]	training's rmse: 0.211971
[1000]	training's rmse: 0.211868
[1100]	training's rmse: 0.211766
[1200]	training's rmse: 0.211669
[1300]	training's rmse: 0.211576
[1400]	training's rmse: 0.211482
[1500]	training's rmse: 0.21139
[1600]	training's rmse: 0.2113
[1700]	training's rmse: 0.211203
[1800]	training's rmse: 0.211107
[1900]	training's rmse: 0.211014
[2000]	training's rmse: 0.210923
[2100]	training's rmse: 0.210834
Did not meet early stopping. Best iteration is:
[2146]	training's rmse: 0.210793
CPU times: user 16min 54s, sys: 6.38 s, total: 17min 1s
Wall time: 2min 13s


In [22]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_1006BL2.csv.gz', compression='gzip', index=False, header=True)

In [23]:
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.455357
1503425,65b9484d670f,0.118079
1503426,8bab230b2ecd,0.115850
1503427,8e348601fefc,0.071969
1503428,8bd2fe400b89,0.201608
